In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import seabornextends as snsexts
import faker
from faker import utils
from seabornextends import plots, retouch
from seabornextends.retouch import grid, ax, fig

In [ ]:
sns.set_style('whitegrid')
mpl.rcParams['grid.color'] = '0.95'
mpl.rcParams['grid.alpha'] = 0.8
mpl.rcParams['font.family'] = 'Roboto Condensed'

# Develop intuitive understanding of common distributions

## Parameters

In [ ]:
# default parameters for this template
# (these are overriden during dynamic notebook execution)
dist_family = 'norm'
kind = 'continuous'

# default shape
default_shape_kws = {'loc': 0, 'scale': 1}

# grid of shapes
shape_grid = {
    'loc': [2, 4, 6],
    'scale': [3, 6, 9]
}

# for plotting
row = 'loc'
col = 'scale'
sharex = True
sharey = True
kde = True

# params for creating evenly spaced probabilities
LO_PROB = 0.001
HI_PROB = 0.999
n_prob_linspace = 1000

# common lo/hi probabilities
lo_p = 0.025
hi_p = 0.975

# list of common probabilities
common_probs = [0.01, 0.025, 0.05, 0.1, 0.4, 0.5, 0.6, 0.9, 0.95, 0.975, 0.99]

## Get distribution function from scipy

In [ ]:
dist_func = getattr(stats, dist_family)
dist_func

## Compare this distribution in different shapes

Generate random samples from this distribution in different shapes (of size `iters` each):

In [ ]:
df = faker.utils.stacked_from_function(func=getattr(dist_func, 'rvs'),
                                       param_grid=shape_grid,
                                       col_names=['random_value'],
                                       iters=10000,
                                       seed=12345)

In [ ]:
grid = snsexts.plots.distplot(a='random_value',
                              data=df,
                              row=row,
                              col=col,
                              sharex=sharex,
                              sharey=sharey,
                              distplot_kws={'hist': True, 'kde': kde})
retoucher = retouch.grid.FacetGridRetoucher(grid)
retoucher.fig.set_dpi(300)

## Instantiate a distribution

In [ ]:
mydist = dist_func(**default_shape_kws)
mydist

In order to make this notebook work for both continuous and discrete distributions
we create a common alias for PDF/PMF.

In [ ]:
if kind == 'continuous':
    pf = mydist.pdf
    pfname = 'PDF'
    pfdesc = 'Probability Density Function'
    logpf = mydist.logpdf
    logpfname = 'Log PDF'
    logpfdesc = 'Log Probability Density Function'
    
elif kind == 'discrete':
    pf = mydist.pmf
    pfname = 'PMF'
    pfdesc = 'Probability Mass Function'
    
    logpf = mydist.logpmf
    logpfname = 'Log PMF'
    logpfdesc = 'Log Probability Mass Function'    

Moments:

In [ ]:
mean, var, skew, kurt = mydist.stats(moments='mvsk')
print("mean: ", mean)
print("var: ", var)
print("skew: ", skew)
print("kurt: ", kurt)

### Generate evenly spaced probabilities

Generate a list of evenly spaced probabilities:

In [ ]:
prob_linspace = np.linspace(start=LO_PROB, stop=HI_PROB, num=n_prob_linspace)
prob_linspace[0:5], prob_linspace[-5:]

Get _quantiles_ corresponding to probabilities in `prob_linspace`

In [ ]:
quantiles = mydist.ppf(prob_linspace)
quantiles[0:5], quantiles[-5:]

## Generate random numbers from this distribution

In [ ]:
rands = mydist.rvs(size=100000)
rands[0:5]

In [ ]:
ax = sns.distplot(rands)
ax.axvline(np.mean(rands), label='mean: {0:.5f}'.format(np.mean(rands)), color='C3')
ax.axvline(np.median(rands), label='median: {0:.5f}'.format(np.median(rands)), color='C3', ls='dashed')
ax.axvline(mydist.mean(), label='mean of frozen dist: {0:.5f}'.format(mydist.mean()), color='C4')
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## Probabilty Density Function / Probability Mass Function

Take a `quantile` as input and return a _point evaluated_ `probability`.

**NOTE**: for normal distribution with mean different to 0 and std different to 1, quantile is a value x (e.g. 1160) _and not_ the standardised version of that value.

To be a **valid pdf/pmf**, a function must satisfy:
* It must be larger than or equal to zero everywhere (a value must be possible).
* The total area under it must be one.

In [ ]:
probs = common_probs
for p in probs:
    print("{n} evaluated at quantile {q:.3f} -- which marks {cdf:.3f} CDF probability in distribution -- is {logpf:.3f} (evaluated at that point only).".format(
        q=mydist.ppf(p),
        n=pfname,
        cdf=mydist.cdf(mydist.ppf(p)),
        logpf=pf(mydist.ppf(p))))

We evaluate a list of quantiles and return their **probability density function estimate** (point-estimates only). 

In the standard normal, quantiles at the extremes of the distribution have very low probability while quantiles in the middle of the distribution have very high probability.

In [ ]:
estimates = pf(mydist.ppf(prob_linspace))

In [ ]:
mydist.ppf(prob_linspace)[0:5], estimates[0:5]

In [ ]:
mydist.ppf(prob_linspace)[-5:], estimates[-5:]

In [ ]:
# plot
fig, ax = plt.subplots(1, 1)

# function
ax.plot(mydist.ppf(prob_linspace),
        pf(mydist.ppf(prob_linspace)),
        lw=3,
        alpha=0.6,
        color='black',
        label='{0} {1}'.format(dist_family, pfname))

# lo_p

ax.vlines(x=mydist.ppf(lo_p),
          ymin=0,
          ymax=pf(mydist.ppf(lo_p)),
          colors='C3',
          alpha=1,
          label='point probability estimate for quantile {q:.3f} is {p:.3f}'.format(p=pf(mydist.ppf(lo_p)),
                                                                                    q=mydist.ppf(lo_p)))

ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= lo_p))]),
                y1=pf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                        (prob_linspace <= lo_p))])),
                color='C3',
                alpha=0.1)

# hi_p
ax.vlines(x=mydist.ppf(hi_p),
          ymin=0,
          ymax=pf(mydist.ppf(hi_p)),
          color='C0',
          alpha=1,
          label='point probability estimate for quantile {q:.3f} is {p:.3f}'.format(p=pf(mydist.ppf(hi_p)),
                                                                                    q=mydist.ppf(hi_p)))


ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= hi_p) &
                                                    (prob_linspace <= HI_PROB))]),
                y1=pf(mydist.ppf(prob_linspace[np.where((prob_linspace >= hi_p) &
                                                        (prob_linspace <= HI_PROB))])),
                color='C0',
                alpha=0.1)

plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## Log Probabilty Density Function / Probability Mass Function

In [ ]:
probs = common_probs
for p in probs:
    print("{n} evaluated at quantile {q:.3f} -- which marks {cdf:.3f} CDF probability in distribution -- is {logpf:.3f} (evaluated at that point only).".format(
        q=mydist.ppf(p),
        n=logpfname,
        cdf=mydist.cdf(mydist.ppf(p)),
        logpf=logpf(mydist.ppf(p))))

In [ ]:
estimates = logpf(mydist.ppf(prob_linspace))

In [ ]:
mydist.ppf(prob_linspace)[0:5], estimates[0:5]

In [ ]:
mydist.ppf(prob_linspace)[-5:], estimates[-5:]

In [ ]:
# plot
fig, ax = plt.subplots(1, 1)

# function
ax.plot(mydist.ppf(prob_linspace),
        logpf(mydist.ppf(prob_linspace)),
        lw=3,
        alpha=0.6,
        color='black',
        label='{0} {1}'.format(dist_family, pfname))

# lo_p

ax.vlines(x=mydist.ppf(lo_p),
          ymin=0,
          ymax=logpf(mydist.ppf(lo_p)),
          colors='C3',
          alpha=1,
          label='point probability estimate for quantile {q:.3f} is {p:.3f}'.format(p=logpf(mydist.ppf(lo_p)),
                                                                                    q=mydist.ppf(lo_p)))

ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= lo_p))]),
                y1=logpf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                        (prob_linspace <= lo_p))])),
                color='C3',
                alpha=0.1)

# hi_p
ax.vlines(x=mydist.ppf(hi_p),
          ymin=0,
          ymax=logpf(mydist.ppf(hi_p)),
          color='C0',
          alpha=1,
          label='point probability estimate for quantile {q:.3f} is {p:.3f}'.format(p=logpf(mydist.ppf(hi_p)),
                                                                                    q=mydist.ppf(hi_p)))


ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= hi_p) &
                                                    (prob_linspace <= HI_PROB))]),
                y1=logpf(mydist.ppf(prob_linspace[np.where((prob_linspace >= hi_p) &
                                                        (prob_linspace <= HI_PROB))])),
                color='C0',
                alpha=0.1)

plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## PPF (Percent Point Function or Quantile Function)

Take a _lower tail_ `probability` as input and return a `quantile` (inverse of `CDF`).

It answers questions such as `What is the value below which 97.5% of distribution falls under?`

In [ ]:
probs = [0.025, 0.5, 0.975]
for p in probs:
     print("Quantile corresponding to {p} lower tail probability is {q:.3f} ({p:.1%} of values are lower than or equal to {q:.3f})".format(
         p=p,
         q=mydist.ppf(p)))

Calculate quantiles corresponding to list of _lower tail_ probabilities:

In [ ]:
estimates = mydist.ppf(prob_linspace)

In [ ]:
prob_linspace[0:5], estimates[0:5]

In [ ]:
prob_linspace[-5:], estimates[-5:]

Quantiles of common lower tail probabilities:

In [ ]:
probs = common_probs
for p in probs:
     print("Quantile corresponding to {p} lower tail probability is {q:.3f} ({p:.1%} of values are lower than or equal to {q:.3f})".format(
         p=p,
         q=mydist.ppf(p)))

In [ ]:
# plot
fig, ax = plt.subplots(1, 1)

# function
ax.plot(mydist.ppf(prob_linspace),
        prob_linspace,
        lw=3,
        alpha=0.6,
        color='black',
        label='{0} ppf'.format(dist_family))

# lo_p
ax.vlines(x=mydist.ppf(lo_p),
          ymin=0,
          ymax=lo_p,
          colors='C3',
          alpha=1,
          label='quantile for which {p:.1%} values are likely to be lower than or equal to is {q:.3f}'.format(
              p=lo_p,
              q=mydist.ppf(lo_p)))




ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= lo_p))]),
                y1=mydist.cdf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                                (prob_linspace <= lo_p))])),
                color='C3',
                alpha=0.1)

# hi_p
ax.vlines(x=mydist.ppf(hi_p),
          ymin=0,
          ymax=hi_p,
          color='C0',
          alpha=1,
          label='quantile for which {p:.1%} values are likely to be lower than or equal to is {q:.3f}'.format(
              p=hi_p,
              q=mydist.ppf(hi_p)))




ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= hi_p))]),
                y1=mydist.cdf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                                (prob_linspace <= hi_p))])),
                color='C0',
                alpha=0.1)

plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## ISF (Inverse Survival Function)

Take an _upper tail_ `probability` as input and returns a `quantile` (inverse of `SF`).

It answers questions such as `What is the value above which 97.5% of distribution falls above?`

In [ ]:
probs = [0.025, 0.5, 0.975]
for p in probs:
     print("Quantile corresponding to {p} upper tail probability is {q:.3f} ({p:.1%} of values are greater than {q:.3f})".format(
         p=p,
         q=mydist.isf(p)))

Calculate quantiles corresponding to list of _upper tail_ probabilities:

In [ ]:
estimates = mydist.isf(prob_linspace)

In [ ]:
prob_linspace[0:5], estimates[0:5]

In [ ]:
prob_linspace[-5:], estimates[-5:]

Quantiles of common upper tail probabilities:

In [ ]:
probs = common_probs
for p in probs:
     print("Quantile corresponding to {p} upper tail probability is {q:.3f} ({p:.1%} of values are greater than {q:.3f})".format(
         p=p,
         q=mydist.isf(p)))

In [ ]:
# plot
fig, ax = plt.subplots(1, 1)

# function
ax.plot(mydist.isf(prob_linspace),
        prob_linspace,
        lw=3,
        alpha=0.6,
        color='black',
        label='{0} isf'.format(dist_family))

# hi_p
ax.vlines(x=mydist.isf(hi_p),
          ymin=0,
          ymax=hi_p,
          color='C3',
          alpha=1,
          label='quantile for which {p:.1%} values are likely to be greater than is {q:.3f}'.format(
              p=hi_p,
              q=mydist.isf(hi_p)))


ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= hi_p))]),
                y1=mydist.sf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                               (prob_linspace <= hi_p))])),
                color='C3',
                alpha=0.1)

# lo_p
ax.vlines(x=mydist.isf(lo_p),
          ymin=0,
          ymax=lo_p,
          colors='C0',
          alpha=1,
          label='quantile for which {p:.1%} values are likely to be greater than is {q:.3f}'.format(
              p=lo_p,
              q=mydist.isf(lo_p)))



ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= lo_p))]),
                y1=mydist.sf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                               (prob_linspace <= lo_p))])),
                color='C0',
                alpha=0.1)


plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## CDF (Cumulative Density Function)

Take a `quantile` as input and return a `probability`.

CDF returns the `probability` that the random variable is *less than or equal to some value* (a `quantile`)

In [ ]:
probs = common_probs
for p in probs:
    print("Probability that a random variable is *less or equal than* {q:.3f} quantile is {cdf:.3f}.".format(
        q=mydist.ppf(p),
        cdf=mydist.cdf(mydist.ppf(p))))


We evaluate a list of quantiles and return their **cumulative density function** probability. 

In [ ]:
estimates = mydist.cdf(mydist.ppf(prob_linspace))

In [ ]:
mydist.ppf(prob_linspace)[0:5], estimates[0:5]

In [ ]:
mydist.ppf(prob_linspace)[-5:], estimates[-5:]

In [ ]:
# plot
fig, ax = plt.subplots(1, 1)

# function
ax.plot(mydist.ppf(prob_linspace),
        mydist.cdf(mydist.ppf(prob_linspace)),
        lw=3,
        alpha=0.6,
        color='black',
        label='{0} cdf'.format(dist_family))

# lo_p

ax.vlines(x=mydist.ppf(lo_p),
          ymin=0,
          ymax=mydist.cdf(mydist.ppf(lo_p)),
          colors='C3',
          alpha=1,
          label='probability that a random variable\n from this distribution is lower than quantile {q:.3f} is {p:.1%}'.format(
              p=mydist.cdf(mydist.ppf(lo_p)),
              q=mydist.ppf(lo_p)))



ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= lo_p))]),
                y1=mydist.cdf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                                (prob_linspace <= lo_p))])),
                color='C3',
                alpha=0.1)

# hi_p
ax.vlines(x=mydist.ppf(hi_p),
          ymin=0,
          ymax=mydist.cdf(mydist.ppf(hi_p)),
          color='C0',
          alpha=1,
          label='probability that a random variable\n from this distribution is lower than quantile {q:.3f} is {p:.1%}'.format(
              p=mydist.cdf(mydist.ppf(hi_p)),
              q=mydist.ppf(hi_p)))



ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                    (prob_linspace <= hi_p))]),
                y1=mydist.cdf(mydist.ppf(prob_linspace[np.where((prob_linspace >= LO_PROB) &
                                                                (prob_linspace <= hi_p))])),
                color='C0',
                alpha=0.1)

plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## SF (Survival Function)

Take a `quantile` as input and return a `probability`.

SF returns the `probability` that the random variable is *greater than some value* (a `quantile`)

In [ ]:
probs = common_probs
for p in probs:
    print("Probability that a random variable is *greater than* {q:.3f} quantile is {sf:.3f}.".format(
        q=mydist.ppf(p),
        sf=mydist.sf(mydist.ppf(p))))


We evaluate a list of quantiles and return their **survival function** probability. 

In [ ]:
estimates = mydist.sf(mydist.ppf(prob_linspace))

In [ ]:
mydist.ppf(prob_linspace)[0:5], estimates[0:5]

In [ ]:
mydist.ppf(prob_linspace)[-5:], estimates[-5:]

In [ ]:
# plot
fig, ax = plt.subplots(1, 1)

# function
ax.plot(mydist.ppf(prob_linspace),
        mydist.sf(mydist.ppf(prob_linspace)),
        lw=3,
        alpha=0.6,
        color='black',
        label='{0} sf'.format(dist_family))

# lo_p

ax.vlines(x=mydist.ppf(lo_p),
          ymin=0,
          ymax=mydist.sf(mydist.ppf(lo_p)),
          colors='C3',
          alpha=1,
          label='probability that a random variable\n from this distribution is greater than quantile {q:.3f} is {p:.1%}'.format(
              p=mydist.sf(mydist.ppf(lo_p)),
              q=mydist.ppf(lo_p)))



ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= lo_p) &
                                                    (prob_linspace <= HI_PROB))]),
                y1=mydist.sf(mydist.ppf(prob_linspace[np.where((prob_linspace >= lo_p) &
                                                               (prob_linspace <= HI_PROB))])),
                color='C3',
                alpha=0.1)

# hi_p
ax.vlines(x=mydist.ppf(hi_p),
          ymin=0,
          ymax=mydist.sf(mydist.ppf(hi_p)),
          color='C0',
          alpha=1,
          label='probability that a random variable\n from this distribution is greater than quantile {q:.3f} is {p:.1%}'.format(
              p=mydist.sf(mydist.ppf(hi_p)),
              q=mydist.ppf(hi_p)))



ax.fill_between(x=mydist.ppf(prob_linspace[np.where((prob_linspace >= hi_p) &
                                                    (prob_linspace <= HI_PROB))]),
                y1=mydist.sf(mydist.ppf(prob_linspace[np.where((prob_linspace >= hi_p) &
                                                               (prob_linspace <= HI_PROB))])),
                color='C0',
                alpha=0.1)

plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.gcf().set_dpi(150)

## Variance of sample means

We treat the sample mean as a random variable in itself which has its own distribution. Central limit theorem shows that the mean of this distribution converges towards the population mean from which we drew the sample as the sample size increases.

In [ ]:
n_samples = 1000
sample_size = 16

### Sampling distribution of the mean for a _single_ distribution shape

Draw `n_samples` random samples of size `sample_size`:

In [ ]:
data = faker.utils.from_function(func=mydist.rvs,
                                 func_kws={'size': sample_size},
                                 func_seed_arg='random_state',
                                 iters=n_samples)

print("data.shape: ", data.shape)
data[0:2]

Calculate the mean for each sample drawn:

In [ ]:
sample_means = np.mean(data, axis=1)
print("sample_means.shape: ", sample_means.shape)
sample_means[0:10]

Plot the distribution of sample means

In [ ]:
ax = sns.distplot(sample_means)
ax.axvline(mydist.ppf(0.5), color='C3', lw=2, label='most likely value in mydist')
plt.gcf().set_dpi(150)
plt.legend(bbox_to_anchor=(1, 1), borderaxespad=0.0)

### Sampling distribution of the mean for multiple distribution shapes

In [ ]:
df = faker.utils.stacked_from_function(
    func=getattr(dist_func, 'rvs'),
    func_seed_arg='random_state',
    apply_func=np.mean,
    param_grid=dict(size=[sample_size], **shape_grid),
    col_names=['sample_mean'],
    iters=n_samples,
    seed=12345)
df.head()

In [ ]:
grid = snsexts.plots.distplot(a='sample_mean',
                       data=df,
                       row=row,
                       col=col,
                       sharex=sharex,
                       sharey=sharey,
                       distplot_kws={'hist': True, 'kde': kde})

retoucher = retouch.grid.FacetGridRetoucher(grid)
retoucher.fig.set_dpi(300)
retoucher.fig.set_title(title='Sampling distribution of the mean for multiple distribution shapes', y=1.05)

### Sampling distribution of the mean for a single distribution shape but different sample sizes

In [ ]:
df = faker.utils.stacked_from_function(
    func=getattr(dist_func, 'rvs'),
    func_seed_arg='random_state',
    apply_func=np.mean,
    param_grid=dict(size=[10, 30, 100, 300, 500, 1000],
                    **{k: [v] for k, v in default_shape_kws.items()}),
    col_names=['sample_mean'],
    iters=n_samples,
    seed=12345)
df.head()

In [ ]:
grid = snsexts.plots.distplot(a='sample_mean',
                       data=df,
                       col='size',
                       col_wrap=3,
                       distplot_kws={'hist': True})

retoucher = retouch.grid.FacetGridRetoucher(grid)
retoucher.fig.set_dpi(300)
retoucher.fig.set_title(title='Sampling distribution of the mean for different sample sizes\nfor {}'.format(default_shape_kws), y=1.05)